first 40
hrefs    a   class Card-title
content  div class group
time          itemprop="datePublished"
header   div in href a 

In [1]:
from ipynb.fs.defs.functions import htmltotext, some_actions, query_url, send_query, save_data, check_new_articles

In [11]:
from time import gmtime, strftime
from datetime import datetime, timedelta
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import nltk
import json
import time
import random
import traceback
import pickle

browser = webdriver.Chrome(  executable_path = "C:\\Chrome\\chromedriver" )
browser.set_window_size(1920, 1080)
action  = webdriver.ActionChains(browser)

# Process

In [3]:
def confirm_not_robot():
    print("error for confirm")

In [4]:
def get_article_urls_from_html( soup, articles ):
    
    yesterday = datetime.now() - timedelta(1)
    yesterday = yesterday.strftime("%Y/%m/%d")
    today     = strftime("%Y/%m/%d", gmtime())
    today_yestуrday_date = [yesterday, today]

    articles_url_name = {}
    
    
    for article in articles:
        href = article['href']
        
        # in these domain only video, without text        
        if "com/video" not in href:
            # and article must be only or today created or yesterday            
            if today_yestуrday_date[0] in href or today_yestуrday_date[1] in href :
                # article name                
                if article.find("div"):
                    articles_url_name[ href ] = article.find("div").get_text()
        
    return articles_url_name

In [5]:
def get_urls_from_cotegory( url, cnt=None ): # cnt for recursion count

    html = query_url(url, browser)
    soup = BeautifulSoup(html, 'html.parser')
    
    # a   class Card-title
    articles = soup.find_all( "a", attrs={"class" : "Card-title"} )
    
    # if in captcha page
    if   len( articles ) == 0 and cnt == None :
        confirm_not_robot()
        return get_urls_from_cotegory( url, cnt=1 )
    elif len( articles ) == 0 and cnt == 1:
        return {}
    
    
    # get urls and article names from html    
    return get_article_urls_from_html( soup, articles )

In [6]:
def get_article_content( url, cnt=None ):
    
    html = query_url(url, browser)
    soup = BeautifulSoup(html, 'html.parser')
    
    if  len( soup.find_all( attrs={"class" : "group"} ) ) == 0 and cnt == None :
        confirm_not_robot()
        return get_article_content( url, cnt=1 )
            
    elif len( soup.find_all( attrs={"class" : "group"} ) ) == 0 and cnt == 1:
        return "", ""
    
    # get free content ( group )
    content  = soup.find_all( attrs={"class" : "group"} )

    text_cont = " ".join( [ element.get_text() for element in content ] )  
    text_cont = htmltotext( text_cont )
    
    # get article time
    time = soup.find( attrs={"itemprop" : "datePublished"} ).get_text()
    
    return text_cont, time

In [7]:
def main_process( category, category_articles_data, sleep_between_url , url, url_server ):
    
    # check new articles    
    articles_url_name_new = get_urls_from_cotegory( url )
    articles_old_urls = category_articles_data["url"].values
    new_articles_urls = check_new_articles( list( articles_url_name_new.keys() ) , articles_old_urls )
    
    # if appear new article. get content
    for article_url in new_articles_urls:
        
        text_cont, time_art = get_article_content( article_url )
        
        # add new article data
        new_data = {
            "url"      : article_url,
            "title"    : articles_url_name_new[article_url],
            "text"     : text_cont,
            "date"     : time_art,
            "category" : category
        }
        # send data to server
        try:
            send_query( url_server, new_data )
        except:
            pass
        
        category_articles_data = category_articles_data.append( new_data, ignore_index=True )
        # each 5 article pickle
        if category_articles_data.shape[0] % 10 == 0:
            pickle.dump( category_articles_data , open(pickle_obj_path,"wb")) 
        # if article counts > 300, save 150 
        if category_articles_data.shape[0] > 300:
            category_articles_data = save_data( category_articles_data, df_save_path )
            pickle.dump( category_articles_data , open(pickle_obj_path,"wb")) 
        
        time.sleep( sleep_between_url )
    pickle.dump( category_articles_data , open(pickle_obj_path,"wb")) 

In [14]:
categorys = [ "/markets/", "/business/", "/investing/", "/technology/", "/politics/", "/pro/news/"]
df_columns = ["url", "title", "text", "date", "category"]

url_server = "http://localhost:9000/"
source_url = "https://www.cnbc.com"
pickle_obj_path = "cnbc_data.pkl"
df_save_path = "./CNBC/cnbc_data"

sleep_between_url = 4
sleep_between_cat = 7

In [9]:
category_articles_data = pd.DataFrame( columns = df_columns )

In [ ]:
category_articles_data = pickle.load(open(pickle_obj_path, "rb"))

In [ ]:
while True:

    for category in categorys:

        url = source_url + category

        category_articles_data = pickle.load(open(pickle_obj_path, "rb"))
        main_process( category, category_articles_data, sleep_between_url, url, url_server )
        
        time.sleep(sleep_between_cat)

error for confirm
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

error for confirm
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

error for confirm
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

error for confirm
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

error for confirm
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)



Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

error for confirm
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

error for confirm
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

error for confirm
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

error for confirm
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)

error for confirm
Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=92.0.4515.159)



# Save and Load data

In [ ]:
pickle.dump( category_articles_data , open(pickle_obj_path,"wb")) 

In [ ]:
category_articles_data = pickle.load(open(pickle_obj_path, "rb"))